<a href="https://colab.research.google.com/github/tej143123/Designing-SAT3-Soution-UsingGenetic-Algorithms/blob/main/Designing_SAT3_Solution_using_Genetic_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import io
import time
import numpy as np

In [ ]:
t = []
t1 = time.time()

In [ ]:
def GetReverse(literal):
    if literal[0] == '~':
        return literal[1:]
    else:
        return '~' + literal

In [ ]:
# Function to check if the clauses are satisfiable
def is_satisfiable(n_vars, clauses):
    candidates = {frozenset()}

    for clause in clauses:
        temp = set()

        for s in candidates:
            for literal in clause:
                if GetReverse(literal) not in s:
                    temp.add(s | {literal})

        candidates = temp

        if len(candidates) == 0:
            return False

    return True

In [ ]:
# Function to load the case from the input
def load_case(f):
    n_vars, n_clauses = f.readline().split()
    clauses = [f.readline().strip().split(' v ') for _ in range(int(n_clauses))]
    return int(n_vars), clauses

In [ ]:
# Function to evaluate the fitness of a chromosome
def evaluate_fitness(chromosome, clauses):
    count_false = 0
    for clause in clauses:
        satisfied = False
        for literal in clause:
            if literal[0] == '~':
                # Get the index for negation, handling 'X' prefix
                var_index = int(literal[1:].replace('X', '')) - 1
                if not chromosome[var_index]:  # Check if the variable is False
                    satisfied = True
                    break
            else:
                # Get the index for non-negation, handling 'X' prefix
                var_index = int(literal.replace('X', '')) - 1
                if chromosome[var_index]:  # Check if the variable is True
                    satisfied = True
                    break
        if not satisfied:
            count_false += 1
    fitness = np.exp(-count_false)  # Calculate fitness
    return fitness

In [ ]:
# Function to select parents using tournament selection
def select_parents(population, fitness_scores):
    parents = []
    for _ in range(2):  # Select two parents
        tournament = np.random.choice(len(population), size=3, replace=False)
        selected_parent = tournament[np.argmax(fitness_scores[tournament])]
        parents.append(population[selected_parent])
    return parents

In [ ]:
# Function to perform one-point crossover
def crossover(parent1, parent2):
    point = np.random.randint(1, len(parent1))  # Random crossover point
    offspring1 = np.concatenate((parent1[:point], parent2[point:]))
    offspring2 = np.concatenate((parent2[:point], parent1[point:]))
    return offspring1, offspring2

In [ ]:
# Function to perform mutation
def mutate(chromosome, mutation_rate=0.1):
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]  # Flip the bit
    return chromosome

In [ ]:
# Main function
def main(f=sys.stdin):
    num_cases = int(f.readline())
    for _ in range(num_cases):
        n_vars, clauses = load_case(f)
        print(clauses)

        # Initialize population
        pop_size = 10
        population = np.random.randint(0, 2, (pop_size, n_vars))

        # Print the initial population
        print("Initial Population:")
        print(population)

        generations = 100  # Set number of generations
        mutation_rate = 0.1  # Mutation probability
        best_individual = None  # To store the best individual
        best_fitness = 0.0  # To store the best fitness value
        t1 = time.time()  # Start timing

        for generation in range(generations):
            fitness_scores = np.array([evaluate_fitness(individual, clauses) for individual in population])

            # Update the best individual and fitness
            current_best_fitness = np.max(fitness_scores)
            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                best_individual = population[np.argmax(fitness_scores)]

            # Check if a satisfactory solution is found
            if best_fitness == 1.0:
                print(f"Solution found in generation {generation}: {best_individual}")
                break

            new_population = []
            for _ in range(pop_size // 2):
                parents = select_parents(population, fitness_scores)
                offspring1, offspring2 = crossover(parents[0], parents[1])
                new_population.append(mutate(offspring1, mutation_rate))
                new_population.append(mutate(offspring2, mutation_rate))
            population = np.array(new_population)

        print(f"Best fitness in final generation: {best_fitness}")
        print(f"Population corresponding to best fitness: {best_individual}")  # Print the best individual
        print(f"Execution time: {time.time() - t1} seconds")

# Sample data for testing
data = """
1
3 5
X1 v X2 v X3
X1 v ~X2
X2 v ~X3
X3 v ~X1
~X1 v ~X2 v ~X3
""".strip()

# Run the main function with sample data
main(io.StringIO(data))

[['X1', 'X2', 'X3'], ['X1', '~X2'], ['X2', '~X3'], ['X3', '~X1'], ['~X1', '~X2', '~X3']]
Initial Population:
[[1 1 1]
 [0 1 1]
 [1 1 1]
 [0 1 1]
 [1 1 1]
 [0 1 0]
 [1 0 1]
 [1 0 0]
 [0 1 0]
 [1 0 1]]
Best fitness in final generation: 0.36787944117144233
Population corresponding to best fitness: [1 1 1]
Execution time: 0.03663778305053711 seconds
